# Preprocess text

In [2]:
import io
from collections import defaultdict
import os, os.path
import sys
sys.path.insert(0, "..")

import numpy as np
import pandas as pd

import spacy
from spacy.lemmatizer import Lemmatizer
import nl_core_news_lg

from tqdm import tqdm_notebook as tqdm
from pprint import pprint
import spacy
import enchant
from enchant.checker import SpellChecker

from string import punctuation

import torch
import re
import nltk

from difflib import SequenceMatcher
from transformers import AutoTokenizer, AutoModel

from src import iterators

#### Settings

In [3]:
#stopword_list = nltk.corpus.stopwords.words('dutch')
from spacy.lang.nl.stop_words import STOP_WORDS
limit = 0 # limit the data
stopwords = set(STOP_WORDS) # set of stopwords for performance

#### Import the data

In [4]:
# Load csv
csv = iterators.iterate_directory("../data/processed/selected_articles/", ".csv")
df = pd.concat([pd.read_csv(c["article_path"]) for c in csv],ignore_index=True)
df.sort_values(by=["count"], ascending=False, inplace=True)

### Clean the dataset

#### Remove non Chinese, Japanese and Korean char

In [6]:
def jkc_detect(texts):
    # korean
    if re.search("[\uac00-\ud7a3]", texts):
        return False
    # japanese
    if re.search("[\u3040-\u30ff]", texts):
        return False
    # chinese
    if re.search("[\u4e00-\u9FFF]", texts):
        return False
    return True

In [7]:
# Only non jkc
df['non_jkc'] = df['text'].apply(jkc_detect)
df = df[df['non_jkc'] == True]

In [8]:
df.tail(3)

,Unnamed: 0_x,type,text,article_name,date,index_article,article_filepath,dir,Unnamed: 0_y,metadata_title,...,newspaper_title,newspaper_date,newspaper_city,newspaper_publisher,newspaper_source,newspaper_volume,newspaper_issuenumber,newspaper_language,count,non_jkc
51807,128976,p,58. Boksterm. 59. Zangnoot. 60.' Bekende motor...,DDD_010852519_0090_articletext.xml,1950-04-08,2097838,/Users/leonardovida/dev/HistAware/data/raw/del...,/Users/leonardovida/dev/HistAware/data/raw/del...,964.0,DDD:ddd:010852519:mpeg21.didl.xml.gz.xml,...,De waarheid,1950-04-08,Amsterdam,s.n.,Internationaal Instituut voor Sociale Geschied...,9.0,286.0,nl,1,True
51806,128964,p,> LS we het Friesch (Christelijk Nationaal) Da...,DDD_010852519_0071_articletext.xml,1950-04-08,2097835,/Users/leonardovida/dev/HistAware/data/raw/del...,/Users/leonardovida/dev/HistAware/data/raw/del...,964.0,DDD:ddd:010852519:mpeg21.didl.xml.gz.xml,...,De waarheid,1950-04-08,Amsterdam,s.n.,Internationaal Instituut voor Sociale Geschied...,9.0,286.0,nl,1,True
135906,146893,p,06.00 Awake on the wild side. Presentatie: Reb...,DDD_010692171_0382_articletext.xml,1994-02-11,1347868,/Users/leonardovida/dev/HistAware/data/raw/del...,/Users/leonardovida/dev/HistAware/data/raw/del...,246.0,DDD:ddd:010692171:mpeg21.didl.xml.gz.xml,...,De Telegraaf,1994-02-11,Amsterdam,Dagblad De Telegraaf,KB C 98,102.0,32978.0,nl,1,True


#### Create functions for preprocessing

In [9]:
# Keep "." "!" and "?" to define end of sentence.

punctuation = ",/<>;':\"[]\\{}|`~@#$%^&*()_+-="

def remove_punctuation(text):
    """Remove punctuation"""
    no_punct = "".join([c for c in text if c not in punctuation])
    return(no_punct)

In [10]:
df["text_clean"] = df["text"].apply(lambda x: remove_punctuation(x))

In [11]:
def remove_stopwords(text):
    """Remove stopwords as defined by Spacy stopwords"""
    words = "".join([w for w in text if w not in stopwords])
    return words

In [12]:
df["text_clean"] = df["text_clean"].apply(lambda x: remove_stopwords(x))

In [13]:
def remove_numeric(text):
    """Remove numbers"""
    words = ''.join([c for c in text if not c.isdigit()])
    return words

In [14]:
df["text_clean"] = df["text_clean"].apply(lambda x: remove_numeric(x))

In [16]:
#df["text"][1]
df["text_clean"][1]

'Het antwoord van Mr. Dr. Biskool heeft mijn argmenten niet knnen weerleggen hetgeen ik hieronder in het kort zal aantonen. De heer Biskool begint met terg te nemen de door hem geciteerde voorbeelden welke zoden moeten bewijzen dat in Sriname de Staten bij het bepalen van qorm rekening hielden met vacatres. Deze bewijzen zijn hem ontvallen en daarmede ook zijn stelling. De Staten bestonden tot  in de praktijk it  leden ongeacht de mogelijkheid van een groter aantal. Het qorm was . In het Reglement van Orde is dat getal ook genoemd. Het is onjist om te beweren dat er vóór  geen wettelijk aantal leden der Staten bestond. Het Regerings Regiement geeft het wettelijk aantal aan. Het beroep op art.  R. R. regelende de aftreding van leden bij rooster doet vreemd aan want dit staat geheel biten dc kwestie van qorm. Ik zie niet it welke feiten de conclsie moet worden getrokken dat zelfs de wet lees Regerings Reglement bij de berekening van het qorm rekening hodt met vacatres. Hoe komt de heer B

In [17]:
def cleaner(df):
    "Extract relevant text from DataFrame using a regex"
    # Regex pattern for only alphanumeric, hyphenated text with 3 or more chars
    pattern = re.compile(r"[A-Za-z\-]{2,40}")
    df['text_no_point'] = df['text_clean'].str.findall(pattern).str.join(' ')
    if limit > 0:
        return df.iloc[:limit, :].copy()
    else:
        return df

In [20]:
cleaner(df)
df.tail(2)

,Unnamed: 0_x,type,text,article_name,date,index_article,article_filepath,dir,Unnamed: 0_y,metadata_title,...,newspaper_city,newspaper_publisher,newspaper_source,newspaper_volume,newspaper_issuenumber,newspaper_language,count,non_jkc,text_clean,text_no_point
51806,128964,p,> LS we het Friesch (Christelijk Nationaal) Da...,DDD_010852519_0071_articletext.xml,1950-04-08,2097835,/Users/leonardovida/dev/HistAware/data/raw/del...,/Users/leonardovida/dev/HistAware/data/raw/del...,964.0,DDD:ddd:010852519:mpeg21.didl.xml.gz.xml,...,Amsterdam,s.n.,Internationaal Instituut voor Sociale Geschied...,9.0,286.0,nl,1,True,LS we het Friesch Christelijk Nationaal Dagbl...,LS we het Friesch Christelijk Nationaal Dagbla...
135906,146893,p,06.00 Awake on the wild side. Presentatie: Reb...,DDD_010692171_0382_articletext.xml,1994-02-11,1347868,/Users/leonardovida/dev/HistAware/data/raw/del...,/Users/leonardovida/dev/HistAware/data/raw/del...,246.0,DDD:ddd:010692171:mpeg21.didl.xml.gz.xml,...,Amsterdam,Dagblad De Telegraaf,KB C 98,102.0,32978.0,nl,1,True,. Awake on the wild side. Presentatie Rebecca ...,Awake on the wild side Presentatie Rebecca de ...


### Split long text

In [21]:
def get_split(txt, length):
  len_tot = []
  len_partial = []
  if len(txt.split())//length >0:
    n = len(txt.split())//length
  else: 
    n = 1
  for w in range(n):
    if w == 0:
      len_partial = txt.split()[:length]
      len_tot.append(" ".join(len_partial))
    else:
      len_partial = txt.split()[w*length:w*length + length]
      len_tot.append(" ".join(len_partial))
  return len_tot

In [22]:
df["text_clean_split"] = df["text_clean"].apply(get_split, length=500)
df["text_clean_split_no_point"] = df["text_no_point"].apply(get_split, length=500)

### Save cleaned df

In [23]:
df.to_csv("cleaned_df.csv")

----

### Additional subsetting only to relevant part

Select the divided text and retrieve also article_ids and article_name

In [154]:
divided_texts = []
idx_texts = []
name_texts = []
dfids_texts = []
for idx, row in df.iterrows():
  for text in row['text_split']:
    divided_texts.append(text)
    idx_texts.append(idx)
    name_texts.append(row["article_name"])
    dfids_texts.append(row["Unnamed: 0_x"])

Create smaller dataframe for analysis

In [155]:
df_texts = pd.DataFrame({"text":divided_texts, "article_id":dfids_texts, "article_name":name_texts})
df_texts.head(5)

,text,article_id,article_name
0,Aoiang er mijnen bestaan is het mijngas de gro...,36288,DDD_010417712_0100_articletext.xml
1,ning tijdens de ontgassing normaal kan doorgaa...,36381,DDD_010417712_0102_articletext.xml
2,"""W/ij eijn deze keer op een Joodse bruiloft, ""...",107454,DDD_010612570_0079_articletext.xml
3,Het is een spannende geschiedenis met de gasvo...,122625,DDD_010417601_0094_articletext.xml
4,"In elk geval, meende de archivaris, heeft pate...",125000,DDD_011199673_0059_articletext.xml
